# **<span style="color: black; font-size:1em;">PrOMMiS LCA Integration: Environmental Life Cycle Assessment Using PrOMMiS Modeling Results</span>**

## <span style="color:black; font-weight:bold"> Introduction </span> 

<span style = "color:black">
This document demonstrates the application of Life Cycle Assessment (LCA) methodology to the optimized results from the PrOMMiS model.  
This Jupyter Notebook presents an example LCA application to the West Kentucky No.13 Coal Refuse Flowsheet optimized in the PrOMMiS model.
</br>

<u> Goal and Scope:</u> In this Jupyter Notebook, we use life cycle assessment to evaluate the environmental impact of Rare Earth Oxide Recovery from coal mining refuse.</br>
As shown in the system boundary below, the scope of this work starts with the leaching of size-reduced REE-rich feedstock (REE: Rare Earth Elements) and ends with the recovery of mixed REO solids. The process consists of six main stages: 1) Mixing and Leaching, 2) Rougher Solvent Extraction, 3) Cleaner Solvent Extraction, 4) Precipitation, 5) Solid-Liquid (S/L) separation, and 6) Roasting. </br>
It's crucial to note here that the current script does not account for:

* Upstream processes leading to the production of REE-rich feedstock 


* Downstream processes leading to the separation of REE contained in the REO    

<u> Main Product:</u> The main product of the flowsheet modeled in this Jupyter Notebook is Rare Earth Oxide (REO) solid.

<u> Co-products or by-products:</u> None

<u> Functional Unit:</u> The function unit is 1 kg of recovered REO solids

<u> Allocation:</u> The evaluated flowsheet produces a single product with no intermediate co-products or by-product. As a result, there is no need for allocation in this modeling exercise.

<u> Life cycle inventory Estimation:</u> The material and energy inputs shown in the system boundary figure below have been shortlisted and estimated based on the UKy flowsheet output, [technical reports on the production on the production of REE from coal and coal by-products](https://www.osti.gov/servlets/purl/1569277), and other relevant literature.
</span>

<span style = "color:black">

**Useful notes, definitions, and links:**
- **PrOMMiS model:** Process Optimization and Modeling for Minerals Sustainability initiative, led by the U.S. Department of Energy (DOE), specifically under NETL (National Energy Technology Laboratory). The PrOMMiS source code can be accessed on github through the following repository link: [https://github.com/prommis/prommis/tree/main](https://github.com/prommis/prommis/tree/main)<br> PrOMMiS serves to optimize processing flowsheets. This Jupyter Notebook focuses on a flowsheet for the extraction of REE from coal refuse. 


- **UKy Flowsheet Code:** [https://github.com/prommis/prommis/blob/main/src/prommis/uky/uky_flowsheet.py](https://github.com/prommis/prommis/blob/main/src/prommis/uky/uky_flowsheet.py)

</span>

<div style="text-align: center;">
    <img src="images/system_boundary_1.png" width="1000"/>
</div>

## <span style="color:black; font-weight:bold"> Step 1: Import the necessary tools </span> 

In [ ]:
# import main libraries
import pandas as pd
import olca_schema as olca

from netlolca.NetlOlca import NetlOlca

import prommis.uky.uky_flowsheet as uky # module to run the PrOMMiS model
import src as lca_prommis

## <span style="color:black; font-weight:bold"> Step 2: Run PrOMMiS Optimization Model for the UKy flowsheet and extract model results  </span> 

<span style="color:black">  

In the next cell, the main() [function](https://github.com/prommis/prommis/blob/main/src/prommis/uky/uky_flowsheet.py#L255) runs the UKy flowsheet. This step includes:

* Building and connecting the unit model blocks present in the UKy REE processing plant.
* Setting the scaling factors
* Setting the operating conditions
* Initializing the values for all streams in the system
* Solving the system

This function returns a model object which is used to extract all the PrOMMiS results needed to estimate the flowsheet material and energy inputs.

The [get_lca_df()](https://github.com/KeyLogicLCA/lca-prommis/blob/main/src/prommis_LCA_data.py#L72) function extracts the main PrOMMiS results to calculate material and energy flows needed to develop a life cycle assessment model.
The inventory estimation is primarily based on the results reported by the PrOMMiS model with a few exceptions:
* The oxalic acid input rate is estimated based on the known input of sulfuric acid, and applying the ratio of Oxalic acid to sulfuric acid as reported in the literature for similar processes.
* Reducing agent and caustic solution inputs are reported in [similar flowsheets](https://www.osti.gov/servlets/purl/1569277) for the solvent extraction stages. However, the input amounts are not reported in the UKy flowsheet in PrOMMiS, and as such are omitted in this analysis due to the lack of data and information to estimate them. 
</br>

</span>

In [ ]:
# Run UKy flowsheet, and get prommis model
m, _ = uky.main() 

# Extract LCA data
prommis_data = lca_prommis.data_lca.get_lca_df(m)
prommis_data.to_csv("output/lca_df.csv")

## <span style="color:black; font-weight:bold"> Step 3: Review PrOMMiS Results </span> 

In [ ]:
prommis_data

## <span style="color:black; font-weight:bold"> Step 4: Organize, Categorize, and Covert PrOMMiS Results to LCA Relevant Flows </span> 

<span style = "color:black">

**Purpose:** This step converts the raw PrOMMiS model results into Life Cycle Assessment (LCA) relevant units.

The [convert_flows_to_lca_units()](https://github.com/KeyLogicLCA/lca-prommis/blob/main/src/prommis_LCA_conversions.py#L121) function processes the PrOMMiS data extracted in Step 2 and performs several key unit changes:

- Converts all flow rates and mass/mole fractions from the PrOMMiS model (which reports streams in kg/hr, mol/hr, etc. and components by mass or mole fraction) into equivalent quantities (e.g., mass or volume).
- Converts molar flows to mass flows using molecular weights when mol_to_kg=True using the pubchempy and pymatgen libraries
- Standardizes water flows to a specified unit (m3, L, or kg)

<u><b>Calculation example:</u></b> If the flow rate is 10 kg/hr and the mass fraction of 'Input 1' is 20%, the mass of 'Input 1' is calculated as 10*20/100 = <b>4.536 kg</b>. All the inventory is estimating on the basis of a 1-hour timestep.   

Key Parameters:

- ```hours=1```: Time period for conversion (1 hour based on PrOMMiS model timestep).</br> 
<u>Important note on timestep selection</u>: After converting values into LCA-relevant units, all emissions, as well as material and energy inputs, are normalized to the functional unit: in this case, 1 kg of REO product. This is done by dividing each input and emission by the amount of REO produced. In doing so, the influence of timestep selection is removed. Therefore, the chosen timestep has no impact on the final exchanges table used to build the openLCA model. 
- ```mol_to_kg=True```: Converts molar flows to mass flows using molecular weights
- ```water_unit='m3'```: Standardizes water flows to cubic meters. Other options are 'L' and 'kg' (kg may cause errors when assigning a unit in openLCA)

The function creates a new DataFrame with two additional columns:

- **LCA_Amount:** The amount for a flow in the new LCA unit
- **LCA_Unit:** The standardized unit for LCA modeling

This converted data is saved as ```output/lca_df_converted.csv```


</span>

In [ ]:
# Hours is the time period for the conversion
# mol_to_kg is a boolean that indicates whether to convert moles to kg
# water_unit is the unit of water (m3 or L recommended, kg is also an option)
df = lca_prommis.convert_lca.convert_flows_to_lca_units(prommis_data, hours=1, mol_to_kg=True, water_unit='m3')
df.to_csv('output/lca_df_converted.csv', index=False)

# This will create a new datafrane and csv file that contains the initial prommis_data df with two additional columns:
# 'LCA Amount' and 'LCA Unit'

## <span style="color:black; font-weight:bold"> Step 5: Normalize Flows to the Selected Functional Unit and Review Final LCA Flows </span> 

<span style = "color:black">

**Purpose:** This step transforms the converted PrOMMiS model results into a standardized format suitable for Life Cycle Assessment (LCA) modeling by normalizing flows into a functional unit, merging identical flows, and organizing the data according to LCA conventions.

This cell outlines the steps taken in the [main() function in finalize_LCA_flows.py](https://github.com/KeyLogicLCA/lca-prommis/blob/main/src/finalize_LCA_flows.py#L90). This code makes the following changes to the LCA data:

- Merges the following flows into single flows:
    - Merges the solid feed flows into a singular REO feed input flow
    - Merges the solid output flows into a singular REO product flow
    - Merges wastewater flows into a single flow for liquid waste
    - Merges solid waste flows into a single flow for solid waste
- Normalizes all flows based on the reference flow, 1 kg 73.4% REO Product
- Creates a new dataframe with columns matching openLCA format:
    - Adds a new *category* column for all flows
    - Adds new *context* and *UUID* columns for elementary flows using FEDEFL

This converted data is saved as ```output/lca_df_finalized.csv```


</span>

In [ ]:
# The below code will normalize the converted prommis data to a functional unit
# Note: This code is developed for the UKy flowsheet and the functional unit is automatically set to 1 kg of REO (combination of all REEs)
# TODO: This code should be developed for the other flowsheets. To achieve this, the user should be able to specify the desired functional unit
df = pd.read_csv('output/lca_df_converted.csv')

# Run the merge_flows function for the feed
REO_list = [
    "Yttrium Oxide",
    "Lanthanum Oxide",
    "Cerium Oxide",
    "Praseodymium Oxide",
    "Neodymium Oxide",
    "Samarium Oxide",
    "Gadolinium Oxide",
    "Dysprosium Oxide",
]
df = lca_prommis.final_lca.merge_flows(df, merge_source='Solid Feed', new_flow_name='374 ppm REO Feed', value_2_merge=REO_list)
# This 374 ppm value is directly calculated from the flowsheet. The original study actually used 357 ppm as the feed concentration.

# Run the merge_flows function for the product
df = lca_prommis.final_lca.merge_flows(df, merge_source='Roaster Product', new_flow_name='73.4% REO Product')

# Run the merge_flows function for the liquid waste flows
df = lca_prommis.final_lca.merge_flows(df, merge_source='Wastewater', new_flow_name='Wastewater', merge_column='Category') 
# Note: some of these streams are organic waste, but they're treated as wastewater

# Run the merge_flows function for the solid waste flows
df = lca_prommis.final_lca.merge_flows(df, merge_source='Solid Waste', new_flow_name='Solid Waste', merge_column='Category') 

# Run the finalize_df function
try:
    finalized_df = lca_prommis.final_lca.finalize_df(
        df=df, 
        reference_flow='73.4% REO Product', 
        reference_source='Roaster Product',
        water_type='raw fresh water'
    )
    
    # Get summary
    summary = lca_prommis.final_lca.get_finalize_summary(finalized_df)
    print("Summary:")
    for key, value in summary.items():
        if key != 'flow_type_breakdown':
            print(f"  {key}: {value}")
    
    print("\nFlow Type Breakdown:")
    for flow_type, count in summary['flow_type_breakdown'].items():
        print(f"  {flow_type}: {count}")
        
except Exception as e:
    print(f"Error during finalization: {e}")

finalized_df.to_csv('output/lca_df_finalized.csv', index=False)
finalized_df

## <span style="color:black; font-weight:bold"> Step 6: Connect to openLCA </span> 

#### <span style="color:black; font-weight:bold"> Using this Jupyter Notebook with openLCA </span> 

<span style = "color:black">
In the following modeling stages, this Jupyter Notebook creates an openLCA process, converts it to a product system, and calculates the environmental impacts of the flowsheet described in the introduction.  
As a result, please follow these steps before proceeding to the next cell in this notebook:

* Open openLCA on your desktop, prefereable with a version > 2.50

* Download the PrOMMiS openLCA database from EDX (Next Cell). The PrOMMiS database contains all the processes and impact assessment methods needed to build and evaluate the UKy flowsheet in openLCA.

* Import and open the PrOMMiS openLCA database

* Connect to IPC server (Port 8080). Go to Tools > Developer Tools > IPC Server > Connect

</span>
 

#### <span style="color:black; font-weight:bold"> Download PrOMMiS Database from EDX </span> 

<span style = "color:black">
The cell below downloads the PrOMMiS openLCA database from EDX in JSONLD format and save it in a dedicated 'resources' folder in the current working directory. The following cell will require your EDX API key to download the requested database. Note that the resource ID is set in the next cell but it's advised to cross-check that the ID is correct. The database is constantly being improved and updated, and as a result can have a different resource ID.</br>

The PrOMMiS openLCA database includes:

* The life cycle assessment methods evaluating numerous categories including: water consumption, cumulative energy demand, global warming potential, and acidification potential.

* The background data needed to model the coal refuse flowsheet. This data was retrieved from open-source databases including: USLCI, NETL UP library, and literature. Noteworthy, some processes were unavailable in open-source databases and as a result were modeled and icnluded in the PrOMMiS openLCA database using the following approaches:</br>
    * <b><u>Oxalic Acid</b></u>: The inventory for oxalic was retrieved from ecoinvent v3.11 using the followig process: Oxalic acid {RoW}| oxalic acid production | Cut-off, S </br>
    * <b><u>Ascorbic Acid</b></u>: The inventory for oxalic was retrieved from ecoinvent v3.11 using the followig process: Ascorbic acid {GLO}| market for ascorbic acid | Cut-off, S</br>
    * <b><u>DEHPA</b></u>: The foreground data for DEHPA was retrieved from Cao et al.(2023). To model DEHPA using the literature foreground data, we used the following inventory from the ecoinvent database:</br>
        * <b><u>Acetaldehyde</b></u>:The inventory for acetaldehyde was retrieved from ecoinvent v3.11 using the followig process: Acetaldehyde {GLO}| market for acetaldehyde | Cut-off, S </br>
        * <b><u>Phosphorus oxychloride</b></u>: The inventory for phosphorus oxychloride was retrieved from ecoinvent v3.11 using the followig process: Phosphorus oxychloride {GLO}| market for phoshorus oxychloride | Cut-off, S </br>

<b><u>Instructions to open the PrOMMiS openLCA database</b></u>:</br>

* Create an empty database: New Database > From Scratch > Database content: Empty Database
* Open the database you created
* Import the PrOMMiS_LCA_db that you downloaded: Right Click > Import > File > "PrOMMiS_LCA_db" > Overwrite all existing data sets
* The PrOMMiS database uses an openLCA library that is provided by the Federal [LCA Commons](). The required libraries can be downloaded via the following [link] (https://github.com/FLCAC-admin/uslci-content/raw/refs/heads/dev/downloads/U.S._electricity_baseline_v1.2025-06.0.zip).</br>

<b><u>Sources</b></u>:</br>
[1] ecoinvent. (2024, November). ecoinvent version 3.11. Ecoinvent. 
[2] Yuanyu Cao, Liang Li, Ying Zhang, Zengwen Liu, Liqi Wang, Fan Wu, Jing You, Co-products recovery does not necessarily mitigate environmental and economic tradeoffs in lithium-ion battery recycling, Resources, Conservation and Recycling, Volume 188, 2023, 106689, ISSN 0921-3449, https://doi.org/10.1016/j.resconrec.2022.106689.

To verify that you're using the correct resource ID, follow this link: [https:/edx.netl.doe.gov/workspace/resources/prommis-lca-integration](https:/edx.netl.doe.gov/workspace/resources/prommis-lca-integration) 
</span>
 

In [ ]:
# This is the resource ID of the PrOMMiS openLCA database as of September 30, 2025
resource_id = '1146b7de-ffca-4eec-9f0f-96e78488f52f'

# The function below downloads the database and saves it in a 'resources' folder in the current work directory
lca_prommis.import_db.import_db(resource_id)

#### <span style="color:black; font-weight:bold"> Connect to openLCA IPC Server </span> 

In [ ]:
netl = NetlOlca()
netl.connect()
netl.read()

#### <span style="color:black; font-weight:bold"> Import Exchanges Table </span> 

<span style="color:black"> 
To create a process in openLCA we need to import the exchanges table generates in previous steps.
The exchanges table should have the following:

* <u> Flow Name:</u> this will be later used to guide you in creating exchanges
* <u>LCA Amount:</u> This is the amount normalized to 1 kg of the quantitative reference flow. In this case, it is the amount per kg REO product.
* <u>LCA Unit</u>
* <u>Is_input:</u> Determines whether the created exchange is an input or an output
* <u>Reference Product:</u> There should be one reference product. For example, in this case its the REO Product.
* <u>Flow Type</u>
* <u>Category:</u> Flows are labeled as Technosphere flows (referred to as product flows in openLCA), waste flows, and elementary flows.
* <u>Context:</u> Elementary flows are assigned a context such as: emission/air, emission/ground, resource/ground, etc.
* <u>UUID:</u> Since elementary flows should be assigned a FEDEFL compliant flow in openLCA, they are automatically assigned a UUID. The UUID is assigned based on matching flow name and context in the Federal Elementary Flow List.
* <u>Description</u>

</span>

In [ ]:
# Save the LCA data produced in previous steps in a dataframe called df
df = pd.read_csv('output/lca_df_finalized.csv')
df.head(20)

#### <span style="color:black; font-weight:bold"> Enter Process Information </span> 

##### <span style="color:black; font-weight:bold"> Unit Process Name </span> 

In [ ]:
process_name = "REO Extraction From Coal Mining Refuse | UKy Flowsheet"

##### <span style="color:black; font-weight:bold"> Unit Process Description </span> 

In [ ]:
process_description = "This process involves the production of a Rare Earth Oxide solid extraction from coal mining refuse. The scope of this work starts with the leaching of size-reduced REE-rich feedstock (REE: Rare Earth Elements) and ends with the recovery of mixed REO solids. The process consists of six main stages: 1) Mixing and Leaching, 2) Rougher Solvent Extraction, 3) Cleaner Solvent Extraction, 4) Precipitation, 5) Solid-Liquid (S/L) separation, and 6) Roasting. This process does not account for upstream processes leading to the production of REE-rich feedstock nor does it account for Downstream processes leading to the separation of REE contained in the REO. The main product is a rare earth oxide solid with no other by-products or co-products. The functional unit is 1 kg of recovered REO solids. The material and energy inputs shown in the system boundary figure below have been shortlisted and estimated based on the UKy flowsheet output, as well as other relevant literature."

##### <span style="color:black; font-weight:bold"> Create process with exchanges </span> 

<span style="color:black">  
In the next step, the function reads the exchanges table and examines each row:
</br>

* If the exchange is a reference product, the user can: 1) create an exchange using an existing flow in the PrOMMiS database or 2) create a new flow, create an exchange for it, and set it as a quantitative reference (e.g., reference product)
</br>

* If the exchange is an elementary flow, the function automatically creates an exchange for it using the given uuid.
</br>

* If the exchange is a technosphere or waste flow, the user has to select a flow and and provider in order to create an exchange, using the following sequence:
    1) Enter keyword to search for a flow
    2) Select a flow from a provided list. The function automatically creates a list of all providers generating this flow.
    3) Select a provider from the provided list.

    
</span>

In [ ]:
### create process usign the create_new_process function
process = lca_prommis.create_lca.create_new_process(netl,df,process_name,process_description)
# get the process uuid to be used in the following step when create a product system
process_id = process.id
# NOTE: For future discussion - is it better to select the flow then the provider (current status) or pick provider first then select relevant flows?

##### <span style="color:black; font-weight:bold"> Create product system </span> 

<span style="color:black">  
In the following cell, the created process is used to generate a product system and build the supply chain.
</span>

In [ ]:
# create product system
ps = lca_prommis.create_ps.create_ps(netl, process_id)
# get product system id to use in next step 
ps_uuid = ps.id

##### <span style="color:black; font-weight:bold"> Conduct analysis </span> 

<span style="color:black">  
To conduct the analysis, we need to define three main elements:

* <u>The client</u>: In this case it is netl created in previous cells
* <u>The product system</u>: The cell above creates a product system and extracts its Universally Unique Identifier (UUID)
* <u>The impact assessment method</u>: The PrOMMiS openLCA database is downloaded in the beginning of step 6. This database contains an impact assessment method that encloses the following impact cateogories:
    * <i>Acidification Potential</i>: Imported from TRACI 2.1
    * <i>Cumulative Energy Demand</i>: developed by compiling all the characterization factors from the Federal LCA Commons CED method
    * <i>Cumulative Energy Demand - Non-renewable</i>: Developed by only including the characterization factors for non-renewable resources
    * <i>Cumulative Energy Demand - renewable</i>: Developed by only including the characterization factors for renewable resources
    * <i>Eutrophication Potential</i>: Imported from TRACI 2.1
    * <i>Global Warming Potential [AR6, 100 yr]</i>: Imported from TRACI 2.1
    * <i>Global Warming Potential [AR6, 20 yr]</i>: Imported from TRACI 2.1
    * <i>Human Health - cancer</i>: Imported from TRACI 2.1
    * <i>Human Health - non-cancer</i>: Imported from TRACI 2.1
    * <i>Human Health - particulate matter</i>: Imported from TRACI 2.1
    * <i>Ozone depletion</i>: Imported from TRACI 2.1
    * <i>Smog Formation</i>: Imported from TRACI 2.1
    * <i>Water Consumption (NETL)</i>: Developed by NETL - imported from TRACI 2.1 (NETL)
    
>>>><i><b>NOTE</b></i>: The impact assessment method is defined in the cell below using its UUID. The PrOMMiS openLCA database, and consequently the impact assessment method enclosed in it, might change due to ongoing improvement. It's advised to check the impact assessment method UUID in openLCA before running the next cell.
</span>

In [ ]:
# Impact asessment method UUID
impact_method_uuid = '60cb71ff-0ef0-4e6c-9ce7-c885d921dd15'
# Generate result. The outcome of the run_analysis is a result object which 
# will be used in the following cells to extract total and contribution tree results.
result = lca_prommis.run_analysis.run_analysis(netl, ps_uuid, impact_method_uuid)

##### <span style="color:black; font-weight:bold"> Generate Results - Total Environmental Impact </span> 

<span style="color:black">  
In the next cell, the generate_total_results extracts the total environmental impact from the 'result' object for each impact category.
</span>

In [ ]:
# use the generate_total_results function to generate the total environmental impacts
# this also stores the results in a csv stored in the output folder
result.wait_until_ready()
total_impacts = lca_prommis.generate_total_results.generate_total_results(result)
total_impacts

##### <span style="color:black; font-weight:bold"> Generate Results - Contribution Tree </span> 

<span style="color:black">  
In the following cell, the generate_contribution_tree function extracts the contribution by category.
The contribution tree has two main aspects:

* <i><u>The maximum number of nodes</u></i>: The number of nodes reflects the number of child/constituting nodes to be expanded (e.g., electricity, heat, sulfuric acic, etc.). <i><u><b>Note:</b></u></i> Setting the number of nodes to (-1) returns all the nodes possible <i>(Recommended)</i>.
</br>

* <i><u>The maximum number of levels</u></i>: The number of levels reflects the number of steps away from the main product. <i><u><b>Note:</b></u></i> Setting the number of levels to (1) reports the total impact for each node without any further remifications.


><i><b>NOTE</b></i>: If you set download_results to True, the contribution tree results will be downloaded and can be accessed through the 'output' folder in the current working directory.

</span>

In [ ]:
# Input the number of nodes
max_expand_nodes = int(input("Please enter the maximum number of nodes: "))

# Input the number of levels
max_expand_levels = int(input ("Please enter the maximum number of levels: "))

# Generate the contirbution results
lca_prommis.generate_contribution_tree.generate_contribution_tree(result, max_expand_levels, max_expand_nodes, download_results = True)

##### <span style="color:black; font-weight:bold"> Plot Results </span> 

In [ ]:
lca_prommis.plot_results.plot_results(result)